In [65]:
import numpy as np
import copy
import pandas as pd
from numpy.linalg import solve, norm,cond
from scipy.linalg import hilbert

In [74]:
def qr(a): #алгоритм QR-разложения методом вращений
    n = a.shape[0]
    q, r = np.identity(n), copy.copy(a)
    for i in range(n):
        for j in range(i+1,n):
            c = r[i,i]/(r[i,i]**2 + (r[j,i]**2))**0.5
            s = r[j,i]/(r[i,i]**2 + (r[j,i]**2))**0.5
            r[i,:], r[j,:] = c*r[i,:] + s*r[j,:], -s*r[i,:] + c*r[j,:]
            q[:,i], q[:,j] = c*q[:,i] + s*q[:,j], -s*q[:,i] + c*q[:,j]
    return q,r

In [39]:
def qr_solve(q,r,b=None): #решение СЛАУ QR-методом
    if b is None:
        b = np.random.uniform(-100,100,size=(q.shape[0]))
    n = r.shape[1]
    x, y = np.zeros(n), np.transpose(q) @ b
    for j in range(len(y)):
        x[n-j-1]=(y[n-j-1]-sum([r[n-j-1,n-p-1]*x[n-p-1] for p in range(j)]))/r[n-j-1,n-j-1]
    return x

Проверим, что решения, полученные qr-методом, совпадают с решением системы:

In [53]:
a = np.random.rand(2,2)
b = np.random.rand(2)

In [54]:
q,r = qr(a)
norm(solve(a,b)-qr_solve(q,r,b))

1.3092278833360675e-16

In [55]:
matrixes = [hilbert(n) for n in range(3,6)]

In [68]:
def regularisation_solution(a,b=None):
    if b is None:
        b = np.random.uniform(-100,100,size=(a.shape[1]))
    ans = pd.DataFrame(columns=["alpha","cond(a+alpha*E)","||x-x_alpha||"])
    q,r = qr(a)
    x = qr_solve(q,r,b)
    ans = ans.append(pd.Series([0,cond(a),0],index=ans.columns),True)
    E = np.identity(a.shape[0])
    for i in range(2,13,2):
        a_i = a + 10**(-i)*E
        q,r = qr(a_i)
        x_i = qr_solve(q,r,b)
        ans = ans.append(pd.Series([10**(-i),cond(a_i),norm(x_i-x)],index=ans.columns),True)
    return ans

In [69]:
regularisation_solution(matrixes[0])

,alpha,cond(a+alpha*E),||x-x_alpha||
0,0.000000e+00,524.056778,0.000000
1,1.000000e-02,111.790091,10631.235462
2,1.000000e-04,505.291334,483.909268
3,1.000000e-06,523.862213,5.017296
4,1.000000e-08,524.054831,0.050191
5,1.000000e-10,524.056758,0.000502
6,1.000000e-12,524.056777,0.000005


In [72]:
regularisation_solution(matrixes[1])

,alpha,cond(a+alpha*E),||x-x_alpha||
0,0.000000e+00,15513.738739,0.000000
1,1.000000e-02,149.575003,14748.748061
2,1.000000e-04,7627.334553,7413.777308
3,1.000000e-06,15354.963172,149.254833
4,1.000000e-08,15512.134730,1.507826
5,1.000000e-10,15513.722697,0.015080
6,1.000000e-12,15513.738579,0.000151


In [73]:
regularisation_solution(matrixes[2])

,alpha,cond(a+alpha*E),||x-x_alpha||
0,0.000000e+00,476607.250242,0.000000e+00
1,1.000000e-02,157.653234,2.763984e+07
2,1.000000e-04,15172.641273,2.676870e+07
3,1.000000e-06,365456.558250,6.448061e+06
4,1.000000e-08,475162.081827,8.383694e+04
5,1.000000e-10,476592.755044,8.408939e+02
6,1.000000e-12,476607.105284,8.409333e+00
